In [1]:
import zipfile
with zipfile.ZipFile("dataset-resized.zip", 'r') as zip_ref:
    zip_ref.extractall("trashnet")

In [29]:
from torchvision.models import resnet34,ResNet34_Weights
import torch.nn as nn
import torch
from torchsummary import summary
import torch.optim as optim
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Dataset loading

In [6]:
sample_img = Image.open("trashnet/dataset-resized/plastic/plastic1.jpg")
sample_img_arr = np.array(sample_img)
H,W,_ = np.shape(sample_img)

In [7]:
classes = ["cardboard","glass","metal","paper","plastic","trash"]
class_sizes = {"cardboard":403,"glass":501,"metal":410,"paper":594,"plastic":482,"trash":137}
class_nums =  {"cardboard":0,"glass":1,"metal":2,"paper":3,"plastic":4,"trash":5}

#n_whole = np.sum(list(class_sizes.values()))
n_whole = 30*6

In [8]:
X = np.zeros((n_whole,H,W,3))
y = np.zeros((n_whole))


In [9]:
n_whole

180

In [11]:
#update X and y
total_count  = 0
for categ in classes:
    n_elem = class_sizes[categ]
    n_elem = 30
    for i in range(1,1+n_elem):
        if total_count%100 == 0:
            print(f'total_count = {total_count}')
        X[total_count] = np.array(Image.open(f'trashnet/dataset-resized/{categ}/{categ}{i}.jpg'))
        y[total_count] = class_nums[categ]
        total_count += 1

total_count = 0
total_count = 100


In [12]:
X = X.reshape((n_whole,3,H,W))

In [13]:
train_X,test_X,train_y,test_y = train_test_split(X, y, test_size=15, random_state=42)
val_X,test_X ,val_y,test_y = train_test_split(test_X, test_y, test_size=5, random_state=42)

In [14]:
train_X = torch.FloatTensor(train_X)
train_y = torch.LongTensor(train_y)
train_dataset = torch.utils.data.TensorDataset(train_X, train_y)
trainloader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=64, shuffle=True, num_workers=2)

#not sure we would uuse validationloader or testloader
val_X = torch.FloatTensor(val_X)
val_y = torch.LongTensor(val_y)
val_dataset = torch.utils.data.TensorDataset(val_X, val_y)
valloader = torch.utils.data.DataLoader(dataset=val_dataset, batch_size=64, shuffle=True, num_workers=2)

test_X = torch.FloatTensor(test_X)
test_y = torch.LongTensor(test_y)
test_dataset = torch.utils.data.TensorDataset(test_X, test_y)
testloader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=64, shuffle=True, num_workers=2)

In [15]:
dataloaders = {'train':trainloader,'val':valloader,'test':testloader}


## model initialization 

In [16]:
model = resnet34()
model.load_state_dict(torch.load("model_weights.pth"))

<All keys matched successfully>

In [17]:
first_layer = list(model.children())[0]
print(f' first layer: {first_layer}')

 first layer: Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)


In [18]:
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 6)

model = model.to(device)
summary(model,(3,H,W))
# 512 x 384 x 3

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 192, 256]           9,408
       BatchNorm2d-2         [-1, 64, 192, 256]             128
              ReLU-3         [-1, 64, 192, 256]               0
         MaxPool2d-4          [-1, 64, 96, 128]               0
            Conv2d-5          [-1, 64, 96, 128]          36,864
       BatchNorm2d-6          [-1, 64, 96, 128]             128
              ReLU-7          [-1, 64, 96, 128]               0
            Conv2d-8          [-1, 64, 96, 128]          36,864
       BatchNorm2d-9          [-1, 64, 96, 128]             128
             ReLU-10          [-1, 64, 96, 128]               0
       BasicBlock-11          [-1, 64, 96, 128]               0
           Conv2d-12          [-1, 64, 96, 128]          36,864
      BatchNorm2d-13          [-1, 64, 96, 128]             128
             ReLU-14          [-1, 64, 

In [19]:
# freeze all layers
for param in model.parameters():
    param.requires_grad = False
last_layer = list(model.children())[-1]
print(f'except last layer: {last_layer}')
for param in last_layer.parameters():
    param.requires_grad = True

except last layer: Linear(in_features=512, out_features=6, bias=True)


In [20]:
criterion = nn.CrossEntropyLoss()
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [68]:
score = 0
outputs = None
for epoch in range(100):  # loop over the dataset multiple times

    running_loss = 0.0
    model.train()
    i = 0
    for inputs, labels in dataloaders['train']:
        # print(i)
        # get the inputs; data is a list of [inputs, labels]
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer_ft.zero_grad()
        # forward + backward + optimize
        # print(inputs[0]) 
        # print(inputs.dtype)
        outputs = model(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer_ft.step()
        
        # print statistics
        running_loss += loss.item()
        score = torch.mean((torch.Tensor.argmax(outputs,axis = 1)==labels).type(torch.FloatTensor))
        i += 1
    if epoch % 5 == 4:    # print every 2000 mini-batches
        print(f'[{epoch + 1}, {i + 1:5d}] loss: {loss.item() / 10:.3f} accuracy: {score}')
        running_loss = 0.0
    exp_lr_scheduler.step()
print('Finished Training')

[5,     4] loss: 0.143 accuracy: 0.5405405163764954
[10,     4] loss: 0.151 accuracy: 0.4864864945411682
[15,     4] loss: 0.155 accuracy: 0.4324324429035187
[20,     4] loss: 0.148 accuracy: 0.5405405163764954
[25,     4] loss: 0.157 accuracy: 0.4324324429035187
[30,     4] loss: 0.148 accuracy: 0.5405405163764954
[35,     4] loss: 0.158 accuracy: 0.37837839126586914
[40,     4] loss: 0.149 accuracy: 0.5135135054588318
[45,     4] loss: 0.153 accuracy: 0.4324324429035187
[50,     4] loss: 0.160 accuracy: 0.4054054021835327
[55,     4] loss: 0.156 accuracy: 0.45945945382118225
[60,     4] loss: 0.161 accuracy: 0.37837839126586914
[65,     4] loss: 0.156 accuracy: 0.4864864945411682
[70,     4] loss: 0.157 accuracy: 0.4054054021835327
[75,     4] loss: 0.162 accuracy: 0.3243243098258972
[80,     4] loss: 0.157 accuracy: 0.4324324429035187
[85,     4] loss: 0.153 accuracy: 0.3513513505458832
[90,     4] loss: 0.149 accuracy: 0.5135135054588318
[95,     4] loss: 0.154 accuracy: 0.45945945

In [12]:
import numpy as np
arr = np.array([501,594,403,482,410,137])
200*arr/np.sum(arr)


array([39.65176098, 47.01226751, 31.89552829, 38.14800158, 32.44954491,
       10.84289672])

In [69]:
1/6

0.16666666666666666